## Conceotnet for Synonyms

In [ ]:
import requests
def get_related_words_conceptnet(word, limit=10):
    url = f"http://api.conceptnet.io/query?node=/c/en/{word}&rel=/r/RelatedTo&limit={limit}"
    response = requests.get(url)
    data = response.json()
    related_words = set()  # Use a set to avoid duplicates

    for edge in data['edges']:
        start_label = edge['start']['label']
        end_label = edge['end']['label']

        if start_label.lower() != word.lower():
            related_words.add(start_label)
        if end_label.lower() != word.lower():
            related_words.add(end_label)

    return list(related_words)

In [ ]:
get_related_words_conceptnet('racism')

['rasizam',
 'inherent',
 'prejudice',
 'ku klux klan',
 'racist',
 'supremacist',
 'racialism',
 'ethnocentrism',
 'multiculturalism']

In [ ]:
racism_list = ['racism','prejudice',
 'supremacist',
 'inherent',
 'racist',
 'ethnocentrism',
 'racialism',
 'multiculturalism']

In [ ]:
racism_list = ['racism','racist','racialism']

In [ ]:
get_related_words_conceptnet('xenophobia')

['ξενοφοβία',
 'ksenofobia',
 'muukalaisviha',
 'ксенофобия',
 'xénophobie',
 'xenofobie',
 'ksenofobija',
 'xenophobic',
 'xenofobia',
 'じょうい']

In [ ]:
xenophobia_list = ['xenophobia','xenophobic','xenofobia','alien','foreigner']

In [ ]:
# Creating a dictionary
keywords_dict = {1: racism_list, 2: xenophobia_list}

print(keywords_dict)

{1: ['racism', 'racist', 'racialism'], 2: ['xenophobia', 'xenophobic', 'xenofobia', 'alien', 'foreigner']}


## Request for Reddit API

In [ ]:
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 5.1 MB/s eta 0:00:00


In [ ]:
import praw

# Initialize a Reddit instance
reddit = praw.Reddit(
    client_id='wIAcnZsvpxA8Rscpr5fkBw',
    client_secret='obTWUUc7RdsvGyppcZWv8yDUnz9rqQ',
    user_agent='SentimentAnalysisBot:v1.0 (by /u/Shayan)',
    check_for_async=False
)

# Choose a subreddit that is generally active, such as 'test'
subreddit = reddit.subreddit('test')

print(f"Fetching new posts in r/{subreddit.display_name}...\n")
# Retrieve and print the top 5 new posts
for post in subreddit.new(limit=5):
    print(f"Title: {post.title}, URL: {post.url}")


Fetching new posts in r/test...

Title: try try try, URL: https://www.reddit.com/r/test/comments/1cwj56s/try_try_try/
Title: asdf, URL: https://www.reddit.com/r/test/comments/1cwj3bh/asdf/
Title: hello, URL: https://www.reddit.com/r/test/comments/1cwib5f/hello/
Title: dd, URL: https://www.reddit.com/r/test/comments/1cwhugs/dd/
Title: test, URL: https://www.reddit.com/r/test/comments/1cwdskr/test/


In [ ]:
# Get list of subreddit names
subreddit_names = [subreddit.display_name for subreddit in reddit.subreddits.popular(limit=None)]

# Print subreddit names
for name in subreddit_names:
    print(name)

Home
AskReddit
NoStupidQuestions
BaldursGate3
facepalm
interestingasfuck
Damnthatsinteresting
LivestreamFail
pics
Palworld
AmItheAsshole
mildlyinfuriating
Piracy
PeterExplainsTheJoke
funny
AITAH
movies
Helldivers
gaming
worldnews
leagueoflegends
pcmasterrace
Unexpected
news
politics
wallstreetbets
todayilearned
nottheonion
memes
PublicFreakout
Wellthatsucks
explainlikeimfive
OutOfTheLoop
OnePiece
BlackPeopleTwitter
buildapc
HonkaiStarRail
SipsTea
Minecraft
mildlyinteresting
nfl
BeAmazed
DIY
nba
MapPorn
Steam
Overwatch
Genshin_Impact
classicwow
soccer
Eldenring
badroommates
personalfinance
antiwork
anime
wow
DnD
technology
WhitePeopleTwitter
EscapefromTarkov
unpopularopinion
popculturechat
videos
BestofRedditorUpdates
youtube
legaladvice
ffxiv
sysadmin
MadeMeSmile
CombatFootage
relationship_advice
discordapp
pcgaming
Games
ChatGPT
GlobalOffensive
2007scape
formula1
CuratedTumblr
Gamingcirclejerk
TikTokCringe
PiratedGames
techsupport
shitposting
theydidthemath
malelivingspace
WTF
cyberpu

In [ ]:
import pandas as pd
subreddits = ["news", "worldnews", "politics"]



# Function to search posts and fetch comments      ###100   10
def search_posts(subreddits, keywords_dict, post_limit=1, comment_limit=1):
    results = []
    for key, keywords in keywords_dict.items():
        for keyword in keywords:
            for subreddit in subreddits:
                for submission in reddit.subreddit(subreddit).search(keyword, limit=post_limit):
                    submission.comments.replace_more(limit=0)  # This line limits the number of "MoreComments" objects to handle
                    comments = [comment.body for comment in submission.comments.list()[:comment_limit]]
                    for comment in comments:
                        results.append({
                            "label": key,  # Assigning the key as the label
                            "keyword": keyword,
                            "comment": comment
                        })
    return pd.DataFrame(results)

# Fetch data
data = search_posts(subreddits, keywords_dict)

# Save data to a CSV file
data.to_csv('racism_xenophobia_data_comments_labeled.csv', index=False)

print("Data with labeled comments saved to 'racism_xenophobia_data_comments_labeled.csv'.")

Data with labeled comments saved to 'racism_xenophobia_data_comments_labeled.csv'.
